In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
from scipy.optimize import minimize

# Find the exact solution by hand

class Boltznann_two_spins:
    """Class Boltzmann computes an optimal wave function for
    a given hamiltonian and initial w = a, b, W """
    def __init__(self, hamiltonian, n = 2):
        self.n = n
        self.hamiltonian = hamiltonian    # function which is defined outside
        self.w_optimal = np.nan    # optimal w = a, b, W      
        self.psi_optimal = np.nan    # optimal psi corresponding to the optimal w
        self.min_energy = np.nan    # minimal energy corresponding to the optimal psi
        self.energy_optimization_history = []
        
    ########### PSI #################################
    
    def psi_M(self, S, w):    # Works for complex w, however, runtime errors
        """Returns a component of a wavefunction
        in the S direction
        psi is given in form of an array and works for any number of spins"""
        a, b, W = w[0], w[1], w[2]
        psi = np.exp(np.sum(a * S))
#         print('W ===', len(W))
        for i in range(len(W)):
            Fi = 2 * np.cosh(b[i] + np.sum(W[i] * S))
            psi *= Fi
        return psi

    def compute_psi(self, w):
        """Computes psi for w = a,b,W 
        psi is given in form of an array and works for any number of spins"""
        n = len(w[0])
        s = generate_spins(n)
        psi = np.zeros(2 ** n, dtype = 'complex')
        for i in range(len(psi)):
            psi[i] = self.psi_M(s[i], w)
        return psi
    
    def compute_psi_scipy(self, w_scipy):
        """Computes psi when for a given w_scipy"""
        w = self.convert_w_from_scipy(w_scipy)
        return self.compute_psi(w)
    
    def normalize(self, psi):
        """Returns normalized psi"""
        psi_star = np.transpose(np.conjugate(psi))
        return psi / np.sqrt(np.sum(psi_star @ psi))
        
    def show_psi(self, psi):
        """psi is a vector with lenght 2^n"""
        s = 'psi = '
        accuracy = '%.2f'
        for i in range(len(psi)):
            basis = bin(i)
            while len(basis) < len(bin(len(psi))) - 1:
                basis = basis[0:2] + '0' + basis[2:]
            if np.real(psi[i]) >= 0:
#                 s += '(' + str('%.5f' % np.real(psi[i])) + ' + i' + str('%.5f' % np.imag(psi[i])) + ')|' + basis[2:] + '>' + ' + '
                s += '(' + str(accuracy % np.real(psi[i])) + ' + ' + str(accuracy % np.imag(psi[i])) + 'i)|' + basis[2:] + '>' + ' + '
            else:
                s += '(' + str(accuracy % np.real(psi[i])) + ' - ' + str(accuracy % np.imag(-psi[i])) + 'i)|' + basis[2:] + '>' + ' + '
        print(s[:-3])
    
    def show_psi_optimal(self):
        """Shows optimal psi"""
        if self.psi_optimal is np.nan:
            raise ValueError('The optimal psi is not computed yet')
        self.show_psi(self.psi_optimal)
    
    ########### PSI #################################
    ########### OMEGA ###############################
    
    def find_len_a(self, N):
        """Returns the size of a for a given length 
        N of w = a,b,W in a form of 1D array"""
        l = 0
        while 2 * l + l ** 2 != N:
            l += 1
        return l
    
    def convert_w_to_scipy(self, w):
        """Converts w = a, b, W into scipy form"""
        a, b, W = w[0], w[1], w[2]
        w_scipy = np.zeros(2 * (len(a) + len(b) + len(W) * len(W[0]))) # w_scipy = [real(a[0]), imag(a[0]), real(a[1]), ...]
        for i in range(len(a)):
            w_scipy[2 * i] = np.real(a[i])
            w_scipy[2 * i + 1] = np.imag(a[i])
        for i in range(len(b)):
            w_scipy[2 * (len(a) + i)] = np.real(b[i])
            w_scipy[2 * (len(a) + i) + 1] = np.imag(b[i])
        for i in range(len(W)):
            for j in range(len(W[0])):
                w_scipy[2 * (len(a) + len(b) + i * len(W) + j)] = np.real(W[i][j])
                w_scipy[2 * (len(a) + len(b) + i * len(W) + j) + 1] = np.imag(W[i][j])
        return w_scipy
    
    def convert_w_from_scipy(self, w_scipy):
        """From w in scipy form returns w = a, b, W"""
        w = np.zeros(len(w_scipy) // 2, dtype = 'complex')
        for i in range(len(w)):
            w[i] = w_scipy[2 * i] + 1j * w_scipy[2 * i + 1]
        l = self.find_len_a(len(w))
        a, b = w[:l], w[l : 2 * l]
        W = np.resize(w[2 * l + 1:], (l, l))
        return a, b, W
    
    def show_omega_optimal(self):
        """Shows optimal w"""
        w = self.w_optimal
        a, b, W = w[0], w[1], w[2]
        print('a = ', a, '\nb = ', b, '\nW = ', W)
        
    ########### OMEGA ###############################
    ########### ENERGY ##############################
    
    def avg_energy(self, psi):
        """Returns the average energy for a given  wavefunciton
        and a given hamiltonian
        <E> = <psi|H|psi> / <psi|psi>"""
        psi_star = np.transpose(np.conjugate(psi))
        return np.sum(psi_star * self.hamiltonian(psi)) / np.sum(psi_star * psi)
    
    def avg_energy_scipy(self, w_scipy):
        """Returns the average energy for a given configuration w,
        made to work for scipy.optimize.minimize
        Size of w is 2N, even is real, odd is complex"""
        psi = self.compute_psi_scipy(w_scipy)
        self.energy_optimization_history.append(self.avg_energy(psi))
        return np.real(self.avg_energy(psi))
#         return self.avg_energy(psi)
    
    def show_min_energy(self):
        if self.min_energy is np.nan:
            raise ValueError('The optimal energy is not computed yet')
        s = 'E_min = ' + str(self.min_energy)
        print(s)
    
    ########### ENERGY ##############################
    ########### OPTIMIZATION ########################
    
    def find_optimal_psi(self, w0, tolerance = None):
        """For a given hamiltonian searches for the
        ground state, i.e. the psi which minimizes the energy"""
        w0_scipy = self.convert_w_to_scipy(w0)
#         w_min = minimize(self.avg_energy_scipy, w0_scipy)    # x0 is now complex, size 2N, even spots real, odd imag
        w_min = minimize(self.avg_energy_scipy, w0_scipy, tol = tolerance)    # x0 is now complex, size 2N, even spots real, odd imag
        w_optimal_scipy = w_min.x
        self.w_optimal = self.convert_w_from_scipy(w_optimal_scipy)    # w_optimal is in a, b, W form
        self.psi_optimal = self.compute_psi(self.w_optimal)
        self.min_energy = self.avg_energy(self.psi_optimal)
    
    ########### OPTIMIZATION ########################
    

########### NEW HAMILTONIAN #########################

def sigma_x(psi):
    """Applies sum sigma_x_i on a given 
    wavefunction psi (in form of 1D array) and returns the 
    new wavefunciton psi1"""
    n = len(psi)
    psi1 = np.zeros((n), dtype = 'complex')
    for i in range(n):
        # s = 1001 meaning |1001>, iterates through all the possible states 001, 010, 011 ...
        s = bin(i)
        # instead of i = 1 ==> b = 0b1, we want b = 0b001, for 3 spins
        while len(s) < len(bin(n)) - 1:
            s = s[0:2] + '0' + s[2:]
        for j in range(len(s[2:])):
            s1 = ''    # flipping all the possible bits once; 101 would give s1: 100, 111, 001
            for k in s:    # constructing s1 as a function of j               j:  0,   1,   2
                if j - 1 == len(s[2:]) - len(s1):
                    if k == '0':
                        s1 += '1'
                    else:
                        s1 += '0'
                else:
                    s1 += k
            i1 = int(s1, 2)
#             print(s, s1)
#             print(i, i1)
            psi1[i1] += psi[i]
    return psi1

def double_sigma_z(psi):
    """Applies  sigma_z_i sigma_z_{i+1}
    on a given wavefunction psi and returns the 
    new wavefunciton"""
    n = len(psi)
    n_spins = int(np.log2(n))
    psi_sigma_z = np.zeros(len(psi), dtype = 'complex')
    if n == 2:
        for i in range(len(psi)):
            psi_sigma_z[i] = psi[i]
        psi_sigma_z[1] *= -1
        psi_sigma_z[2] *= -1
    else:
        s = generate_spins(n_spins)
        for i in range(n):
            for j in range(n_spins):
                psi_sigma_z[i] += (-1) ** (1 + (s[i][j] + s[i][(j + 1) % len(s[0])]) // 2) * psi[i]
    return psi_sigma_z

def hamiltonian(psi):
    """Retuns wave function after the TFI Hamiltonian 
    acts on it, with h = 1 (to be modified later)"""
#     return h * sigma_x(psi)
    return - h * sigma_x(psi) - double_sigma_z(psi)

def generate_spins(n):
    s = np.zeros((2 ** n, n))
    for i in range(2 ** n):
        b = bin(i)
        while len(b) < n + 2:
            b = b[0:2] + '0' + b[2:]
        d = []
        for k in b:
            if k == '0':
                d.append(-1)
#                     d.append(0)    # This could be more convenient, we'll see
            else:
                d.append(1)
        s[i] = np.array(d[2:])
    return s

########### NEW HAMILTONIAN #########################


In [ ]:
########### INITIALIZATION ########################
n = 5
a = np.random.rand(n)
b = np.random.rand(n)
W = np.random.rand(n,n) + 1j * np.random.rand(n,n)
w = a, b, W
h = 0.4
boltzmann = Boltznann_two_spins(hamiltonian, n)
########### INITIALIZATION ########################

########### OPTIMZATION ###########################
tolerance = 0.003
boltzmann.find_optimal_psi(w, tolerance)
########### OPTIMZATION ###########################

########### EXTRACTING PARAMETERS #################
w_optimal = boltzmann.w_optimal
psi_optimal = boltzmann.psi_optimal
min_energy = boltzmann.min_energy
########### EXTRACTING PARAMETERS #################

In [ ]:
boltzmann.show_psi_optimal()
boltzmann.show_min_energy()
boltzmann.show_omega_optimal()

In [ ]:
plt.plot(np.real(boltzmann.energy_optimization_history))